

First, let's take a look at manually constructing an AST representing "(+ 1 2)". To do this in Python (using Python's internals) we import the `ast` module:

In [2]:
(import "ast")

(ast)

To use the built-in Python tools for interpreting an expression, we must build the AST in a precise manner. So, we take the idea of "(+ 1 2)" and we express this as:

In [3]:
(define expr 
  (ast.Expression
    (ast.BinOp (ast.Num 1)
               (ast.Add)
               (ast.Num 2))))

Because our expr didn't actually come from source code (e.g., from a file or a string) we need to poke in dummy values for line numbers so that the compiler won't complain:

In [4]:
(set! expr (ast.fix_missing_locations expr))

In Python, the AST is compiled into *bytecode*:

In [5]:
(define bytecode (compile expr "<string>" "eval"))

Finally, Python can interpret the bytecode and give you the result:

In [6]:
(python-eval bytecode)

3

# 

In [2]:
(import "ast")

(ast)

In [3]:
(define pyexpr (ast.parse "print(1); print(2); print(3); 4; 5; 6"))

In [4]:
(ast.dump pyexpr)

"Module(body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Num(n=1)], keywords=[], starargs=None, kwargs=None)), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Num(n=2)], keywords=[], starargs=None, kwargs=None)), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Num(n=3)], keywords=[], starargs=None, kwargs=None)), Expr(value=Num(n=4)), Expr(value=Num(n=5)), Expr(value=Num(n=6))])"

In [5]:
(define evaluator
  (lambda (expr)
    (cond
     ((isinstance expr ast.Module)
      (evaluator-begin (vector->list expr.body)))
     ((isinstance expr ast.Expr)
      (evaluator expr.value))
     ((isinstance expr ast.BinOp)
      (evaluator-binop expr.op 
                   (evaluator expr.left)
                   (evaluator expr.right)))
     ((isinstance expr ast.Num)
      expr.n)
     ((isinstance expr ast.Call)
      (evaluator-apply expr.func.id (map evaluator (vector->list expr.args))))
     (else (error "evaluator" "invalid ast: ~s" expr)))))

(define evaluator-apply
  (lambda (op operands)
    (cond
     ((string=? op "print")
      (apply print operands))
     (else (error "evaluator-appy" "unknown apply operator: ~s" op)))))

(define evaluator-begin
  (lambda (exprs)
    (cond
     ((null? exprs)
      (void))
     ((= 1 (length exprs))
      (evaluator (car exprs)))
     (else (begin
            (evaluator (car exprs))
            (evaluator-begin (cdr exprs)))))))

(define evaluator-binop
  (lambda (op left right)
    (cond
     ((isinstance op ast.Add)
      (+ left right))
     ((isinstance op ast.Mult)
      (* left right))
     (else (error "apply-binop" "unknown operator: ~s" op)))))

In [6]:
(evaluator pyexpr)

1
2
3


6

In [8]:
(define codeobj (compile pyexpr "<string>" "exec"))

In [9]:
(define pyeval (getitem __builtins__ "eval"))

In [10]:
(print (pyeval codeobj))

1
2
3
None
